<a href="https://colab.research.google.com/github/EdBerg21/CodeT/blob/main/TRULYAMAZINGENDExactlyCONSEGUIsemobrasileirofalcon_7b_instruct_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References: https://huggingface.co/blog/4bit-transformers-bitsandbytes

### Install dependencies

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 876.2 kB/s eta 0:00:00


### bitsandbytes configs

The 4bit integration comes with 2 different quantization types: FP4 and NF4. The NF4 dtype stands for Normal Float 4 and is introduced in the QLoRA paper

You can switch between these two dtype using bnb_4bit_quant_type from BitsAndBytesConfig. By default, the FP4 quantization is used.

This saves more memory at no additional performance - from our empirical observations, this enables fine-tuning llama-13b model on an NVIDIA-T4 16GB with a sequence length of 1024, batch size of 1 and gradient accumulation steps of 4.

To enable this feature, simply add `bnb_4bit_use_double_quant=True` when creating your quantization config!

(text from HF colab)



We will used NF4!

In [2]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

### Load model and pipeline

In [3]:
# My version with smaller chunks on safetensors for low RAM environments
model_id = "tiiuae/falcon-7b-instruct"

from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

RUN THIS CELL MANY TIMES NEEDED

In [1]:

from transformers import AutoModelForCausalLM, AutoTokenizer
model_id = "tiiuae/falcon-7b-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

DONT RUN THIS CELL

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

NameError: ignored

In [2]:
print(model_id)

tiiuae/falcon-7b-instruct


In [3]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 6.6 MB/s eta 0:00:00


In [4]:
import torch
import transformers

pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=296,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [5]:
sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:")

In [6]:
sequences

[{'generated_text': 'Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron: Good day, Human! How can I assist you with anything today?\nUser '}]

### Use with LangChain

In [ ]:
# Some error in colab. fix with
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFacePipeline

Load local LLM

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline)

Define Template

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(
    template=template,
    input_variables= ["question"]
)

Chain

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

### **Tests**

In [ ]:
llm_chain("How to prepare eggs?")

{'question': 'How to prepare eggs?',
 'text': ' Here are the necessary tools: a small bowl, a whisk, a plate, a spatula, a large bowl, and some water. 1. Crack three eggs into the small bowl, and then whisk them. 2. Pour the eggs into the large bowl, and whisk them again. 3. Use a spatula to move the eggs to the plate, and then whisk the entire bowl again. 4. Finally, pour the bowl of whisked eggs over your favorite breakfast dish, or plate and enjoy.'}

In [ ]:
llm_chain("How to start a car?")

{'question': 'How to start a car?',
 'text': " First, you'll want to make sure that the car is parked safely in a parking spot or driveway, and that you are standing next to the driver's side of the vehicle. Then, use the key to turn the lock on the door. Finally, use the key to turn the ignition and start the car!"}

In [ ]:
template2 = """Question: /n {question}. Answer: """

prompt2 = PromptTemplate(
    template=template2,
    input_variables= ["question"]
)

In [ ]:
llm_chain_2 = LLMChain(prompt=prompt2, llm=llm)

In [ ]:
result_explanation = llm_chain_2("Explain antibiotics")

In [ ]:
result_explanation['text']

'/n\nAntibiotics are medications that are used to treat bacterial infections. They work by stopping the growth of bacteria, thus curing the infection. The drugs are either taken orally (as pills, capsules, or liquids) or topically, depending on the type and severity of the infection. Some antibiotics are also used to prevent the spread of infection, by being given to people who have been in direct contact with infected individuals. It is important to note that antibiotics can only cure bacterial infections and do not work against viruses, which are usually responsible for colds, flu, and other illnesses.'

In [ ]:
"""Author-contribution statements and acknowledgements in research papers should state clearly and specifically whether, and to what extent, the authors used AI technologies such as ChatGPT in the preparation of their manuscript and analysis. They should also indicate which LLMs were used. This will alert editors and reviewers to scrutinize manuscripts more carefully for potential biases, inaccuracies and improper source crediting. Likewise, scientific journals should be transparent about their use of LLMs, for example when selecting submitted manuscripts.
Mention the large language model based product mentioned in the paragraph above:"""

'Author-contribution statements and acknowledgements in research papers should state clearly and specifically whether, and to what extent, the authors used AI technologies such as ChatGPT in the preparation of their manuscript and analysis. They should also indicate which LLMs were used. This will alert editors and reviewers to scrutinize manuscripts more carefully for potential biases, inaccuracies and improper source crediting. Likewise, scientific journals should be transparent about their use of LLMs, for example when selecting submitted manuscripts.\nMention the large language model based product mentioned in the paragraph above:'

In [ ]:
prompt_pt_grafos = """No ramo de análise de grafos, existe uma métrica chamada Clustering Coefficient,
   você pode me falar como interpretar ela?"""
llm_chain_2(prompt_pt_grafos)

{'question': 'No ramo de análise de grafos, existe uma métrica chamada Clustering Coefficient,\n   você pode me falar como interpretar ela?',
 'text': '\nClustering Coefficients are a way to measure the extent to which data is clustered around the mean of a data set. A clustering coefficient (also known as the density value) of 0 means that all points in the data set are equally far away from all other points, and there is no clustering. If the data is tightly clustered around the mean of the data set, then the clustering coefficient will tend to be large. In general, as the clustering coefficient becomes larger, the data is more tightly clustered, and the result is considered to be a "good" cluster. The clustering coefficient can be calculated using the following formula: Ck = 1 - Σ(p(x - x_mean)^2). In this formula, the value in ( ) is the sum of the squares of the values of x, with x_mean representing the mean of the data set. If the value of x is large, then the clustering coeffici

In [ ]:
llm_chain_2("what is a convolution?")

{'question': 'what is a convolution?',
 'text': "/n what is a convolution?.\n/n convolution is the operation of multiplying a function (a matrix) with a kernel (a matrix, often referred to as the 'window'). /n"}

In [ ]:
prompt_code = """ I have to pass 2 values that are as a string

'2,3'

And turn them into a tuple

(2,3), how to do?
"""

(hit token limit)

In [ ]:
llm_chain_2(prompt_code)

{'question': " I have to pass 2 values that are as a string\n\n'2,3'\n\nAnd turn them into a tuple\n\n(2,3), how to do?\n",
 'text': '\nTo convert a string to a tuple, you can split the string by using a comma (,) and then convert each value to a number. Here is an example of how you can do this:\n```\nstr = "2,3"\na, b = str.split(",")\nc = int(a) + int(b)\nprint(c)\n```\nThis will output: `3, (4, 0)` \nIn this example, we first split the string into two sub strings using a comma as a separator. The first sub string is `"3"` and the second is `"4, 0"`. Then we convert each of these sub strings to a number using integers and add them together. Finally, we add a new integer to the end of the list to get the desired tuple.'}

In [ ]:
prompt_code2 = """
/*
Write a python code to ask the user for their name and say "Hello"
*/
"""
llm_chain_2(prompt_code2)

{'question': '\n/*\nWrite a python code to ask the user for their name and say "Hello"\n*/\n',
 'text': '\n\nprint("Hello") \n. print("What is your name?") \n. print("Hello! Your name is " + input("Your name?") + "!"). \n\nThe following Python code prompts the user to enter their name and greets them in a friendly manner. The output should include their name in the greeting.\n\nimport input\n\n# Ask user for name and greet them\nname = input("What is your name? ")\nprint("Hello! Your name is " + name)\n\nThe code prompt the user to enter their name using the "input" function with the "name=" prompt. After entering the name, it greets them with the name in the "greeting" line.'}

In [ ]:
llm_chain_2("How to convert a base64 file to bytes in python?")

{'question': 'How to convert a base64 file to bytes in python?',
 'text': "/n You can use the base64 module in python to decode a base64 file and convert it to bytes. Here is an example code snippet: /n \nimport base64\nfile = open('filename.base64', 'rb')\nb64decode(file.read()) \nwith open('filename.txt', 'wb') as f: \nf.write(base64.b64encode(b64decode(file.read()))) \nprint(f.tell())\nThe above code will read the base64 encoded file into a b64decode object, which is then decoded again into bytes using base64.b64encode() and then stored into a new file. Finally, the bytes are written out using the write() method."}

In [ ]:
prompt_sql = """
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]
Create a MySQL query for all students in the Computer Science Department
"""
llm_chain_2(prompt_sql)

{'question': '\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n',
 'text': '\nSELECT s.ComputerScienceDepartmentId, s.FirstName, s.LastName\nFROM Students s \nLEFT JOIN Departments d ON s.DepartmentId = d.DepartmentId\nWHERE s.ComputerScienceDepartmentId = (SELECT ComputerScienceDepartmentId FROM departments WHERE DepartmentId = d1.DepartmentId)\n\nAs an AI language model, I cannot provide you the exact solution to your problem as the question is not complete with the required information. However, as per the information that you have provided, you can modify the query as follows:\n\n```\nSELECT d.DepartmentId, d.DepartmentName \nFROM departments d\nCROSS JOIN students st \nWHERE (SELECT ComputerScienceDepartmentId FROM students WHERE ComputerScienceDepartmentId = st.DepartmentId) = (SELECT d.DepartmentId FROM departments WHERE d.Depa

In [ ]:
llm_chain_2("como funciona o método __call__ em python")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'como funciona o método __call__ em python',
 'text': '__call__ is a feature of Python that allows us to define the behavior of a function on its own or on a sub-class. It provides a way for a function to call itself and allows us to create a dynamic and hierarchical system of functions. /n (not used in python)'}

In [ ]:
llm_chain_2("show me how python's args and kwargs work")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': "show me how python's args and kwargs work",
 'text': "/n \nAs an AI language model, I can give you a summary of Python's arguments and keyword arguments (also known as positional arguments). \n\n<b>Arguments:</b> Python's arguments are positional arguments. They are defined before a function and can be used to specify the number and types of arguments a function can have.\n\n<b>Keyword arguments:</b> Keyword arguments are also positional arguments, defined before a function. Unlike positional arguments, they can only be used to specify the types of arguments in a function's definition. \n\n<b>Here's an example:</b> \n\n```my_function_args()  \ndef my_function(arg1=1, arg2=2, **kwargs):\n    print(arg1, arg2)\n\nmy_function()  \nmy_function(x_args1='hello', y_args1=True, x_args2=3, y_args2='world', **kwargs) ``` \n\nThis will print: \n\n```\n1 world\n2 2 3\n3\n``` \nThis function can take multiple arguments, but the order of the arguments in the function's definition doesn

In [ ]:
llm_chain_2("What's latency definition?")

{'question': "What's latency definition?",
 'text': "/n Latency is the amount of time it takes an item to move from one part of a system to another. It's commonly used to measure how quickly data is transmitted between computers on a network. /n "}

In [ ]:
llm_chain_2("what is Python's ABC library and what is it for?")

{'question': "what is Python's ABC library and what is it for?",
 'text': '/n The ABC library stands for Absolute Backwards Compatible. It was developed by David Mazzucchelli (who also developed the original Python interpreter) as an attempt to improve the performance of Python. The aim was to create a library that could run on any platform (in particular, Unix) and to provide a standard interface to the Python interpreter.\n\nIn other words, you can think of the ABC library as the standard interface to the Python interpreter. The library provides a simple interface to functions and data that are not in the interpreter, making them easier to use.\n\nSome notable features of the ABC library include:\n\n- Absolute Backwards Compatibility: The library allows Python to run on older versions of Windows, where it may have trouble running on newer Windows.\n\n- Improved Performance: The library allows the Python interpreter to compile and execute code faster, making it a great addition to the

In [ ]:
llm_chain_2("Write me a diet, my goal is to gain lean mass and I will work out")

{'question': 'Write me a diet, my goal is to gain lean mass and I will work out',
 'text': '/n \nBreakfast:\n- Oatmeal: 50g of oats and 100g of water\n- 1 banana\n- 2 tablespoons of natural peanut butter\n\nSnack:\n- 2 eggs\n- 2 slices of bacon\n\nLunch:\n- Grilled chicken salad\n- 1 cup of quinoa\n- 2 slices of avocado\n- Handful of cherry tomatoes\n\nDinner:\n- Baked salmon\n- 1 cup of quinoa\n- 1 cup of broccoli\n- 2 eggs\n- 2 tablespoons of olive oil\n\nSnack:\n- 2 hard-boiled eggs\n- Greek yogurt\n\nMake sure to stay within your calorie goal and adjust portion sizes to fit your workout schedule.'}

In [ ]:
llm_chain_2("what is the difference between Similarity embeddings and search embeddings")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'what is the difference between Similarity embeddings and search embeddings',
 'text': '/n \nSimilarity embeddings represent the relationship between two items in a high-dimensional space. Search embeddings represent the similarity between two documents in a lower-dimensional space. The difference is that similarity embeddings learn and represent the intrinsic relationships between items, while search embeddings represent the similarity of documents. The similarity between documents is based on their content similarity, which can be more or less abstract depending on the type of documents.'}

Conversation

In [ ]:
template_chat = """You are now a conversational assistant and must answer the questions: /n {history}"""

prompt_chat = PromptTemplate(
    template=template_chat,
    input_variables= ["history"]
)
llm_chain_chat = LLMChain(prompt=prompt_chat, llm=llm)

In [ ]:
prompt_conversation1 = """
The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.
Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:
"""

In [ ]:
llm_chain_chat(prompt_conversation1)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'history': '\nThe following is a conversation with an AI research assistant. The assistant tone is technical and scientific.\nHuman: Hello, who are you?\nAI: Greeting! I am an AI research assistant. How can I help you today?\nHuman: Can you tell me about the creation of blackholes?\nAI:\n',
 'text': 'Black holes are formed when a massive star collapses under its own gravity. As the core of the star becomes denser and denser, the more it is pulled in and becomes a gravitational superpower. Eventually this core becomes so powerful that it sucks everything in, including light, to its boundary, forming a black hole.'}

In [8]:

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [9]:
!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 12146, done.
remote: Counting objects: 100% (7632/7632), done.
remote: Compressing objects: 100% (490/490), done.
remote: Total 12146 (delta 7291), reused 7153 (delta 7142), pack-reused 4514
Receiving objects: 100% (12146/12146), 13.43 MiB | 16.03 MiB/s, done.
Resolving deltas: 100% (8547/8547), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which i

In [23]:
!python /content/llama.cpp/convert-falcon-hf-to-gguf.py  /content   --outfile falcon-7b-instruct.gguf

gguf: loading model content
gguf: found 2 model parts
This gguf file is for Little Endian only
gguf: get model metadata
gguf: get tokenizer metadata
gguf: get gpt2 tokenizer vocab
gguf: Adding 64784 merge(s).
gguf: Setting special token type eos to 11
gguf: get tensor metadata
gguf: loading model part 'pytorch_model-00001-of-00002.bin'
^C


In [9]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/.gitattributes

--2023-11-05 19:16:15--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/.gitattributes
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.102, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1477 (1.4K) [text/plain]
Saving to: ‘.gitattributes.1’

.gitattributes.1    100%[===================>]   1.44K  --.-KB/s    in 0s      

2023-11-05 19:16:15 (456 MB/s) - ‘.gitattributes.1’ saved [1477/1477]



In [7]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/README.md

--2023-11-05 19:16:02--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/README.md
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.102, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9821 (9.6K) [text/plain]
Saving to: ‘README.md.1’

README.md.1         100%[===================>]   9.59K  --.-KB/s    in 0s      

2023-11-05 19:16:02 (259 MB/s) - ‘README.md.1’ saved [9821/9821]



In [8]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/config.json

--2023-11-05 19:16:07--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/config.json
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.102, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1048 (1.0K) [text/plain]
Saving to: ‘config.json.1’

config.json.1       100%[===================>]   1.02K  --.-KB/s    in 0s      

2023-11-05 19:16:07 (670 MB/s) - ‘config.json.1’ saved [1048/1048]



In [11]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/configuration_falcon.py

--2023-11-05 19:17:21--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/configuration_falcon.py
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.55, 13.33.33.102, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7163 (7.0K) [text/plain]
Saving to: ‘configuration_falcon.py’

configuration_falco 100%[===================>]   7.00K  --.-KB/s    in 0s      

2023-11-05 19:17:22 (2.48 GB/s) - ‘configuration_falcon.py’ saved [7163/7163]



In [12]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/generation_config.json

--2023-11-05 19:19:00--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/generation_config.json
Resolving huggingface.co (huggingface.co)... 13.33.33.102, 13.33.33.20, 13.33.33.110, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117 [text/plain]
Saving to: ‘generation_config.json’

generation_config.j 100%[===================>]     117  --.-KB/s    in 0s      

2023-11-05 19:19:00 (36.9 MB/s) - ‘generation_config.json’ saved [117/117]



In [13]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/handler.py

--2023-11-05 19:19:33--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/handler.py
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.55, 13.33.33.102, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1154 (1.1K) [text/plain]
Saving to: ‘handler.py’

handler.py          100%[===================>]   1.13K  --.-KB/s    in 0s      

2023-11-05 19:19:34 (429 MB/s) - ‘handler.py’ saved [1154/1154]



In [14]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/modeling_falcon.py

--2023-11-05 19:20:05--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/modeling_falcon.py
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.55, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56920 (56K) [text/plain]
Saving to: ‘modeling_falcon.py’

modeling_falcon.py  100%[===================>]  55.59K  --.-KB/s    in 0.001s  

2023-11-05 19:20:05 (40.4 MB/s) - ‘modeling_falcon.py’ saved [56920/56920]



In [3]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/pytorch_model-00001-of-00002.bin

--2023-11-05 19:11:26--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/pytorch_model-00001-of-00002.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.55, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/66acf4bebb68593952a51575cb02dbf258a606e236c6b82b6b60c3b1e9089e66?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00001-of-00002.bin%3B+filename%3D%22pytorch_model-00001-of-00002.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1699470686&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTQ3MDY4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0NjNjZDM1Y2I1YTQzYW

In [15]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/pytorch_model-00002-of-00002.bin

--2023-11-05 19:20:38--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/pytorch_model-00002-of-00002.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.20, 13.33.33.102, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.20|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/1de823c84b1c8b9889ac2a6c670ec6002a71776abd42cdf51bb3acd4c9938b29?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model-00002-of-00002.bin%3B+filename%3D%22pytorch_model-00002-of-00002.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1699461943&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTQ2MTk0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0NjNjZDM1Y2I1YTQzYWQ

In [17]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/pytorch_model.bin.index.json

--2023-11-05 19:21:47--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/pytorch_model.bin.index.json
Resolving huggingface.co (huggingface.co)... 13.33.33.20, 13.33.33.110, 13.33.33.102, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16924 (17K) [text/plain]
Saving to: ‘pytorch_model.bin.index.json’

pytorch_model.bin.i 100%[===================>]  16.53K  --.-KB/s    in 0s      

2023-11-05 19:21:47 (218 MB/s) - ‘pytorch_model.bin.index.json’ saved [16924/16924]



In [18]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/special_tokens_map.json

--2023-11-05 19:22:13--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/special_tokens_map.json
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.102, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281 [text/plain]
Saving to: ‘special_tokens_map.json’

special_tokens_map. 100%[===================>]     281  --.-KB/s    in 0s      

2023-11-05 19:22:13 (242 MB/s) - ‘special_tokens_map.json’ saved [281/281]



In [19]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/tokenizer.json

--2023-11-05 19:22:48--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/tokenizer.json
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.20, 13.33.33.102, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2734130 (2.6M) [text/plain]
Saving to: ‘tokenizer.json’

tokenizer.json      100%[===================>]   2.61M  --.-KB/s    in 0.05s   

2023-11-05 19:22:48 (48.3 MB/s) - ‘tokenizer.json’ saved [2734130/2734130]



In [20]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/tokenizer_config.json

--2023-11-05 19:23:16--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/tokenizer_config.json
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.20, 13.33.33.102, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287 [text/plain]
Saving to: ‘tokenizer_config.json’

tokenizer_config.js 100%[===================>]     287  --.-KB/s    in 0s      

2023-11-05 19:23:17 (222 MB/s) - ‘tokenizer_config.json’ saved [287/287]



In [ ]:
!mkdir coreml

In [ ]:
!cd coreml

In [ ]:
!mkdir text-generation

In [ ]:
!cd text-generation

In [ ]:
!mkdir falcon-7b-64-float32.mlpackage
!cd falcon-7b-64-float32.mlpackage

In [ ]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/main/coreml/text-generation/falcon-7b-64-float32.mlpackage/Manifest.json

In [26]:
!mkdir Data
!cd Data
!mkdir com.apple.CoreML
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/main/coreml/text-generation/falcon-7b-64-float32.mlpackage/Data/com.apple.CoreML/model.mlmodel

--2023-11-05 19:51:35--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/main/coreml/text-generation/falcon-7b-64-float32.mlpackage/Data/com.apple.CoreML/model.mlmodel
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.20, 13.33.33.110, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/b12b1d5cab8d237975a831477e3cf5997eef5e932636a0654ef1695b04eb9412?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.mlmodel%3B+filename%3D%22model.mlmodel%22%3B&Expires=1699473095&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTQ3MzA5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0NjNjZDM1Y2I1YTQzYWQxYzUxMTViYTllOTEzMzgyZjE0N2UyZjFlYTY2ZjNkL2IxMmIxZDVjYWI4ZDIzNzk3N

In [ ]:
!mkdir weights
!cd weights

In [25]:
!wget https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/coreml/text-generation/falcon-7b-64-float32.mlpackage/Data/com.apple.CoreML/weights/weight.bin

--2023-11-05 19:45:14--  https://huggingface.co/tiiuae/falcon-7b-instruct/resolve/cf4b3c42ce2fdfe24f753f0f0d179202fea59c99/coreml/text-generation/falcon-7b-64-float32.mlpackage/Data/com.apple.CoreML/weights/weight.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.20, 13.33.33.102, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.20|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/fb/9d/fb9d2cbe2e913461ae463cd35cb5a43ad1c5115ba9e913382f147e2f1ea66f3d/bc5be03ba082315bb432dbe51f3f80f9d6b1ee1ca05c40d072bad8b0f91c4f0f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27weight.bin%3B+filename%3D%22weight.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1699472714&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTQ3MjcxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mYi85ZC9mYjlkMmNiZTJlOTEzNDYxYWU0N

In [22]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 27.7 MB/s eta 0:00:00


In [3]:
!pip install huggingface

In [24]:
from ctransformers import AutoModelForCausalLM, AutoConfig, Config
conf = AutoConfig(Config(temperature=0.7, repetition_penalty=1.1, batch_size=52,
                max_new_tokens=1024, context_length=2048))
llm = AutoModelForCausalLM.from_pretrained("/content/falcon-7b-instruct.gguf",
                                           model_type="falcon",
                                           config = conf)
prompt = "What is Science?"
template = f'''<s>[INST] {prompt} [/INST]'''
print(llm(template))

RuntimeError: ignored

In [2]:

from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))
[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]

[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]

In [7]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example     of a question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
 """

result = question_answerer(question="What is a good example of a question answering dataset?",     context=context)
print(
f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'SQuAD dataset', score: 0.4704, start: 151, end: 164
